In [126]:
##Brute-Force, Exhaustive Search and Branch&Bound!

import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import scipy.stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression  
import itertools
import copy
import warnings

In [19]:
## Get Data as a Pandas Dataframe! Make sure to have numerical data only! (pd.get_dummies might help)

file ="C:/Users/Praktikant1/Downloads/nuclear.csv"

data1 = pd.read_csv(file,sep=",").astype(float) #.drop(data.columns[1], axis=1)
data = data1.drop(data1.columns[0], axis=1)

#Y = list(data)[0]
#X = list(data)[1:]

#print(data)

In [ ]:
## exhaustive search - Brute Force. create class here

class BruteForce: 
    
    def __init__(self, Y, X, data):
        self.Y = Y
        self.X = X
        self.data = data
   
    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                #print(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(data[X],data[Y])  
        
        #returns R^2
        s = regressor.score(data[X], data[Y])
        
        # Calculates adjusted R^2
        n = len(data)
        k = len(self.X)
        adj_r2 = 1 - (( (1-r2)*(n-1) )/ (n - k -1))
        
        return (s) 
    
    def search(self):
        solutions = []
        combos = self.powerset(self.X)
        for i in combos:
            solutions.append(self.regress(i, self.Y, self.data))  
        return solutions, combos
    
    def solve(self):
        sol, combos = self.search()
        highest_R2 = combos[np.argmax(sol)]
        R2 = max(sol)
        
        print("higehst R^2 is", R2, "with dependant variables:", highest_R2)
        return highest_R2, R2     

In [ ]:
t, r2 = BruteForce(Y,X,data).solve()

print(len(list(data)))

In [149]:
## this will solve all possible linear regressions and return the one with highest R^2

class ExhaustiveSearch: 
    
    def __init__(self, data, lowerbound, upperbound, adjusted_R2):
        self.data = data
        self.lowerbound = lowerbound
        self.upperbound = upperbound
        self.adjusted_R2 = adjusted_R2
        
        ## Raising Expections and Warnings here
        if len(list(self.data)) >= 15:
            warnings.warn("Dear fellow Caveman, surely you are aware that your dataset exceeds 15 variables and may take a looong time to compute if no bounds are specified, consider setting reasonable bounds to find a quick solution! :-)", Warning)
        if str(type(self.data))  != "<class 'pandas.core.frame.DataFrame'>":
            raise TypeError("Input data must be of type pandas Dataframe ( ་ ⍸ ་ )") 
        if type(self.lowerbound) and type(self.upperbound) != int:
            raise TypeError("Bounds must be integers! ಠ_๏") 
    
    def subset(self, X):
        _set = self.powerset(X)
        subset = []
        for i in _set:
            if self.upperbound >= len(i) >= self.lowerbound:
                subset.append(i)
        return subset
        
    def assign(self):
        XX = list(self.data)
        dic = {}
        for i in XX:
            Y = i
            #X = self.powerset([x for x in XX if x != Y])
            X = self.subset([x for x in XX if x != Y])
            dic[Y] = X
        return dic

    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(self.data[X],self.data[Y])
        
        #returns R^2
        r2 = regressor.score(self.data[X], self.data[Y])
        
        #returns Adjusted R^2 
        n = len(self.data)
        k = len(list(self.data[X]))
        adj_r2 = (1 - ( ((1 - r2)*(n - 1))/(n - k - 1) ))
        
        if self.adjusted_R2 == True:
            return adj_r2
        elif self.adjusted_R2 == False:
            return r2
        else:
            raise ValueError("adjusted_r2 needs to be boolean: True or False (ノಠ益ಠ)ノ")
    
    def search(self):
        dic = self.assign()
        newdic = {}
        for y in dic:
            for x in dic[y]:
                newdic[self.regress(x, y, data)] = y, x
        new={}
        sd = sorted(newdic.items(), reverse=True)
        for k,v in sd:
            new[k] = v
        return new
    
    def solve(self):
        results = self.search()
        highest_R2 = list(results.keys())[0]
        best_vars = results[highest_R2]
        
        print("highest R^2 is", highest_R2, "with dependant/indepedant variables:", best_vars)
        return results

In [150]:
wow = ExhaustiveSearch(data = data, lowerbound = 1, upperbound = 2, adjusted_R2 = True).solve()

highest R^2 is 0.7823640370462159 with dependant/indepedant variables: ('date', ['cost', 't1'])


In [125]:
## Next feature: restricting size, upper and lower bounds for model size!

# wow = exhaustive_search(data = data, R^2 = adj, exp_var = 5, lower_bound = 3, upper_bound = 7).solve()
#print(wow)

In [116]:
## compute adjusted R2 
def adj_r2(r2, data, X):
    n = len(data)
    k = len(X)
    adj_r2 = 1 - (( (1-r2)*(n-1) )/ (n - k -1))
    
    return adj_r2